<h2>Imports</h2>

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

# import metric
from keras.metrics import categorical_crossentropy

# optimization method 
from tensorflow.keras.optimizers import SGD

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
import os
import random
from tqdm import tqdm

import torch
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

<h2>Alexnet model</h2>

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False
# model.fc = nn.Linear(4096, 10)
model.classifier._modules['6'] = nn.Linear(4096, 39)
model.eval()

<h2>Data Loading and Exploration</h2>

In [ ]:
# Number of images for each disease

disease_dict = {}

for plant_disease in os.listdir('Dataset/'):
    disease_dict[plant_disease] = len(os.listdir('Dataset/' + plant_disease))

disease_dict

In [ ]:
# Data loader function

def data_loader(path, batch_size):
    
    transform = transforms.Compose(
        [transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor()]
    )
    
    dataset = datasets.ImageFolder(path, transform = transform)
    
    indices = list(range(len(dataset)))
    
    split = int(np.floor(0.85 * len(dataset)))
    
    validation = int(np.floor(0.70 * split))
    
    np.random.shuffle(indices)
    
    train_indices, validation_indices, test_indices = (
        indices[:validation//16],
        indices[validation//16:split//16],
        indices[split//16:((split//16)+100)]
    )
    
    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=train_sampler
    )
    test_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=test_sampler
    )
    validation_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=validation_sampler
    )    
    
    return dataset, train_loader, validation_loader, test_loader

In [ ]:
dataset, train_loader, validation_loader, test_loader = data_loader(path = 'Dataset/', batch_size = 16)

In [ ]:
# Dataset

dataset

In [ ]:
classes = len(dataset.class_to_idx)
print('Number of classes: ', classes)

In [ ]:
disease_dict = dataset.class_to_idx

diseases = list(disease_dict.keys())
categories = list(disease_dict.values())

In [ ]:
n_epochs = 1
device = "cpu"

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for j in range(n_epochs):
    
    train_loss = []
    
    for inputs, targets in tqdm(train_loader):
        print(inputs.shape)
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        
        output = model(inputs)
        
        loss = criterion(output, targets)
        
        train_loss.append(loss.item())
        print(train_loss[-1])
            
        loss.backward()
        optimizer.step()
        
    train_loss = np.mean(train_loss)
    
    validation_loss = []
    
    for inputs, targets in tqdm(validation_loader):

        inputs, targets = inputs.to(device), targets.to(device)

        output = model(inputs)

        loss = criterion(output, targets)

        validation_loss.append(loss.item())  # torch to numpy world

    validation_loss = np.mean(validation_loss)               